In [ ]:
!pip install -q sagemaker-experiments

In [ ]:
import sagemaker
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'sagemaker-studio-book/chapter10/abalone'

In [ ]:
from datetime import datetime, timedelta, timezone
import json
import os
import re
import uuid
from time import sleep, gmtime, strftime
from threading import Thread

import pandas as pd
import numpy as np

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from botocore.exceptions import ClientError
from sagemaker import image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model

## Getting data

In [ ]:
columns = ['Sex', 'Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight', 'Rings']
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data', names=columns)

In [ ]:
df.head()

In [ ]:
df_processed = df.copy()
df_processed['Rings']=df_processed['Rings'].astype(float)
df_processed['Sex'] = df_processed['Sex'].replace(to_replace=['M', 'F', 'I'], value=[2., 1., 0.])
columns=['Rings', 'Sex', 'Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight']
df_processed = df_processed[columns]

In [ ]:
from sklearn.model_selection import train_test_split
df_build, df_test = train_test_split(df_processed, test_size=0.1, random_state=42, 
                                     shuffle=True, stratify=df_processed['Sex'])
df_train, df_val = train_test_split(df_build, test_size=1/9., random_state=42, 
                                    shuffle=True, stratify=df_build['Sex'])

In [ ]:
df_train.shape, df_val.shape, df_test.shape

In [ ]:
columns_no_target = ['Sex', 'Length', 'Diameter', 'Height', 'WholeWeight', 'ShuckedWeight', 'VisceraWeight', 'ShellWeight']

In [ ]:
os.makedirs('abalone', exist_ok=True)
df_train.to_csv('./abalone/abalone_train.csv', index=False)
df_val.to_csv('./abalone/abalone_val.csv', index=False)
df_test.to_csv('./abalone/abalone_test.csv', index=False)
# df_test[columns_no_target].to_csv('./abalone/abalone_test_no_target.csv', index=False, header=False)

desired_s3_uri = f's3://{bucket}/{prefix}/data'
train_data_s3 = sagemaker.s3.S3Uploader.upload(local_path='./abalone/abalone_train.csv',
                                               desired_s3_uri=desired_s3_uri,
                                               sagemaker_session=sess)
val_data_s3 = sagemaker.s3.S3Uploader.upload(local_path='./abalone/abalone_val.csv',
                                             desired_s3_uri=desired_s3_uri,
                                             sagemaker_session=sess)
test_data_s3 = sagemaker.s3.S3Uploader.upload(local_path='./abalone/abalone_test.csv',
                                              desired_s3_uri=desired_s3_uri,
                                              sagemaker_session=sess)
# sagemaker.s3.S3Uploader.upload(local_path='abalone_test_no_target.csv',
#                                desired_s3_uri=desired_s3_uri,
#                                sagemaker_session=sess)

## Train a ML model to predict `Rings`

In [ ]:
# loading from an existing training job
exp_datetime = '2021-12-18-00-04-35'
jobname = f'abalone-xgb-{exp_datetime}'
xgb=sagemaker.estimator.Estimator.attach(jobname)

In [ ]:
image = image_uris.retrieve(region=region, framework='xgboost', version='1.3-1')

exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
jobname = f'abalone-xgb-{exp_datetime}'

experiment_name = 'abalone-age-prediction'

try:
    experiment = Experiment.create(
        experiment_name=experiment_name, 
        description='Predicting age for abalone based on physical measurements.')
except ClientError as e:
    print(f'{experiment_name} experiment already exists! Reusing the existing experiment.')
    
# Creating a new trial for the experiment
exp_trial = Trial.create(experiment_name=experiment_name, 
                         trial_name=jobname)

experiment_config={
    'ExperimentName': experiment_name,
    'TrialName': exp_trial.trial_name,
    'TrialComponentDisplayName': 'Training'}

train_instance_type = 'ml.m5.xlarge'
train_instance_count = 1
s3_output = f's3://{bucket}/{prefix}/abalone_data/training'

xgb = sagemaker.estimator.Estimator(image,
                                    role,
                                    instance_count=train_instance_count,
                                    instance_type=train_instance_type,
                                    output_path=s3_output,
                                    enable_sagemaker_metrics=True,
                                    sagemaker_session=sess)

xgb.set_hyperparameters(objective='reg:squarederror', num_round=20)

train_input = sagemaker.inputs.TrainingInput(s3_data=train_data_s3, 
                                             content_type='csv')
val_input = sagemaker.inputs.TrainingInput(s3_data=val_data_s3, 
                                           content_type='csv')
data_channels={'train': train_input, 'validation': val_input}

xgb.fit(inputs=data_channels, 
        job_name=jobname, 
        experiment_config=experiment_config, 
        wait=True)

## Deploy the model with data capture

In [ ]:
# optional
exp_datetime = '2021-12-18-00-04-35'
endpoint_name = f'abalone-xgb-{exp_datetime}-2'
print(f'EndpointName: {endpoint_name}')

In [ ]:
##S3 prefixes
data_capture_prefix = f'{prefix}/datacapture'
s3_capture_upload_path = f's3://{bucket}/{data_capture_prefix}'

# exp_datetime = strftime('%Y-%m-%d-%H-%M-%S', gmtime())
ground_truth_upload_path = f's3://{bucket}/{prefix}/ground-truth-data/{exp_datetime}'

# reports_prefix = f'{prefix}/reports'
# s3_report_path = f's3://{bucket}/{reports_prefix}'

##Get the model monitor image
# monitor_image_uri = image_uris.retrieve(framework='model-monitor', region=region)

# print('Image URI:', monitor_image_uri)
print(f'Capture path: {s3_capture_upload_path}')
print(f'Ground truth path: {ground_truth_upload_path}')
# print(f'Report path: {s3_report_path}')

In [ ]:
data_capture_config = DataCaptureConfig(enable_capture=True, 
                                        sampling_percentage=100, 
                                        destination_s3_uri=s3_capture_upload_path)

In [ ]:
# endpoint_name = f'abalone-xgb-{exp_datetime}'
print(f'EndpointName: {endpoint_name}')

predictor = xgb.deploy(initial_instance_count=1,
                       instance_type='ml.m5.large',
                       endpoint_name=endpoint_name,
                       serializer=CSVSerializer(),
                       data_capture_config=data_capture_config)

In [ ]:
# loading from an existing endpoint
predictor = Predictor(endpoint_name=endpoint_name, 
                      sagemaker_session=sess, 
                      serializer=CSVSerializer())

## Establish a persistent load with randomness and ground truth

In [ ]:
def add_randomness(series, probability = 0.1):
    random_rate=(np.random.rand(series.shape[0])<probability).astype(float)
    sigma_scale=0.5
    
    new_series = series * np.random.normal(loc=1, scale=sigma_scale*random_rate, size=series.shape)
    
    if random_rate[0] != 1.:
        # if random_rate for Sex (first cell in random_rate) is not 1,
        # then assign a random value from [0,2].
        new_series[0] = float(np.random.randint(0, 2))
    else:
        new_series[0] = series[0]

    return new_series


def drop_random(series, probability = 0.05):
    random_rate=(np.random.rand(series.shape[0])<probability)
    new_series = series.copy()
    new_series[random_rate]=np.nan
    
    return new_series

def convert_nparray_to_string(series):
    new_series = ','.join([str(i) for i in series])
    new_series = new_series.replace('nan', '')
    
    return new_series
    
def upload_ground_truth(records, ground_truth_upload_path, upload_time):
    records_json = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(records_json)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    sagemaker.s3.S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [ ]:
def generate_load_and_ground_truth():
    gt_records=[]
    for i, row in df_test.iterrows():
        suffix = uuid.uuid1().hex
        inference_id = f'{i}-{suffix}'
        
        gt = row['Rings']
        data = row[columns_no_target].values
#         print(inference_id, data)
        new_data = drop_random(add_randomness(data))
        new_data = convert_nparray_to_string(new_data)
#         print(inference_id, new_data)
        out = predictor.predict(data = new_data, inference_id = inference_id)
#         print(inference_id, gt, out)

        gt_data =  {"groundTruthData": {
                            "data": str(gt), 
                            "encoding": "CSV",
                        },
                        "eventMetadata": {
                            "eventId": inference_id,
                        },
                        "eventVersion": "0",
                    }
#         print(gt_data)
        gt_records.append(gt_data)

    upload_ground_truth(gt_records, ground_truth_upload_path, datetime.utcnow())
    
def generate_load_and_ground_truth_forever():
    while True:
        generate_load_and_ground_truth()

In [ ]:
generate_load_and_ground_truth()

In [ ]:
thread = Thread(target=generate_load_and_ground_truth_forever)
thread.start()

In [ ]:
thread.is_alive()

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get("Body").read().decode("utf-8")

In [ ]:
s3_client = boto3.Session().client("s3")
current_endpoint_capture_prefix = "{}/{}".format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")
print("\n ".join(capture_files))

In [ ]:
capture_file = get_obj_body(capture_files[-1])
print(json.dumps(json.loads(capture_file.split("\n")[-2]), indent=2))

# TODO: tear down data quality baseline and schedule, setup another one with Rings in float.

In [ ]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
model_monitor_prefix = f'{prefix}/data-quality-output' # data-quality-output/baseline/
baseline_results_prefix = f'{model_monitor_prefix}/baseline'

baseline_data_uri = train_data_s3
baseline_results_uri = f's3://{bucket}/{baseline_results_prefix}'

s3_report_path = f's3://{bucket}/{model_monitor_prefix}/reports'

print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))
print(f'Report path: {s3_report_path}')

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=1,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=train_data_s3,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=False,
    logs=False
)

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator

mon_schedule_name = f'abalone-data-monitor-schedule-{exp_datetime}-2'

my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    # record_preprocessor_script=pre_processor_script,
    # post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

In [ ]:
# baseline_results_prefix = f'{prefix}/abalone_data/data-quality-output/baseline'
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Files:")
print("\n ".join(report_files))

## create model quality baseline

In [ ]:
from sagemaker.serializers import CSVSerializer, NumpySerializer
from sagemaker.deserializers import NumpyDeserializer, PandasDeserializer, CSVDeserializer

In [ ]:
predictor_np = Predictor(endpoint_name=endpoint_name, 
                         sagemaker_session=sess,
                         serializer=CSVSerializer(),
                         deserializer=CSVDeserializer())
pred=predictor_np.predict(df_val[columns_no_target].values)

In [ ]:
pred_f = [float(i) for i in pred[0]]

In [ ]:
df_val.head()

In [ ]:
df_val['Prediction']=pred_f

In [ ]:
df_val.head()

In [ ]:
df_val[['Rings', 'Prediction']].to_csv('abalone_val_model_quality_baseline.csv', index=False)
model_quality_baseline_s3 = sagemaker.s3.S3Uploader.upload(local_path='abalone_val_model_quality_baseline.csv',
                                             desired_s3_uri=desired_s3_uri,
                                             sagemaker_session=sess)

In [ ]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput

In [ ]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
model_quality_monitor_prefix = f'{prefix}/model-quality-output' # data-quality-output/baseline/
model_quality_baseline_results_prefix = f'{model_quality_monitor_prefix}/baseline'

model_quality_baseline_results_uri = f's3://{bucket}/{model_quality_baseline_results_prefix}'

model_quality_s3_report_path = f's3://{bucket}/{model_quality_monitor_prefix}/reports'

print('Baseline data uri: {}'.format(model_quality_baseline_s3))
print('Baseline results uri: {}'.format(model_quality_baseline_results_uri))
print(f'Report path: {model_quality_s3_report_path}')

In [ ]:
# Create the model quality monitoring object
my_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=1,
    max_runtime_in_seconds=1800,
    sagemaker_session=sess,
)

# Execute the baseline suggestion job.
# You will specify problem type, in this case Binary Classification, and provide other required attributes.
my_model_quality_monitor.suggest_baseline(
    baseline_dataset=model_quality_baseline_s3,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=model_quality_baseline_results_uri,
    problem_type='Regression',
    inference_attribute='Prediction',
#     probability_attribute="probability",
    ground_truth_attribute='Rings',
    wait=False,
    logs=False
)

## create Schedule

In [ ]:
model_quality_baseline_results_uri

In [ ]:
mon_schedule_name_2 = f'abalone-modelquality-monitor-schedule-{exp_datetime}-2'

# Create an enpointInput
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    inference_attribute='0',
    destination='/opt/ml/processing/input_data',
    start_time_offset='-PT1H',
    end_time_offset='-PT0H'
)
response = my_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name_2,
    endpoint_input=endpointInput,
    output_s3_uri=model_quality_baseline_results_uri,
    problem_type='Regression',
    ground_truth_input=ground_truth_upload_path,
    constraints=my_model_quality_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

In [ ]:
my_model_quality_monitor.baseline_statistics().body_dict

In [ ]:
my_model_quality_monitor.suggested_constraints().body_dict

## Create model bias monitor

In [ ]:
from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

In [ ]:
columns

In [ ]:
context=predictor.endpoint_context()

In [ ]:
context.models()

In [ ]:
model_bias_monitor = ModelBiasMonitor(
    role=role,
    sagemaker_session=sess,
    max_runtime_in_seconds=1800,
)

model_bias_monitor_prefix = f'{prefix}/model-bias-output'
model_bias_baselining_job_result_uri = f's3://{bucket}/{model_bias_monitor_prefix}/baseline'
model_bias_s3_report_path = f's3://{bucket}/{model_bias_monitor_prefix}/reports'

model_bias_data_config = DataConfig(
    s3_data_input_path=val_data_s3,
    s3_output_path=model_bias_baselining_job_result_uri,
    label='Rings',
    headers=columns,
    dataset_type='text/csv',
)

model_bias_config = BiasConfig(
    label_values_or_threshold=[df_train.Rings.median()],
    facet_name='Sex',
    facet_values_or_threshold=[0.0], # 0.0 represents Infant abalone
)

model_predicted_label_config = ModelPredictedLabelConfig()

model_config = ModelConfig(
    model_name='abalone-xgb-2021-12-18-00-04-35-2021-12-29-23-42-52-509',
    instance_count=1,
    instance_type='ml.m5.large',
    content_type='text/csv',
    accept_type='text/csv',
)

In [ ]:
model_bias_monitor.suggest_baseline(
    model_config=model_config,
    data_config=model_bias_data_config,
    bias_config=model_bias_config,
    model_predicted_label_config=model_predicted_label_config,
)
print(f'ModelBiasMonitor baselining job: {model_bias_monitor.latest_baselining_job_name}')

In [ ]:
model_bias_constraints = model_bias_monitor.suggested_constraints()
print()
print(f"ModelBiasMonitor suggested constraints: {model_bias_constraints.file_s3_uri}")
print(S3Downloader.read_file(model_bias_constraints.file_s3_uri))

In [ ]:
model_bias_analysis_config = None
if not model_bias_monitor.latest_baselining_job:
    model_bias_analysis_config = BiasAnalysisConfig(
        model_bias_config,
        headers=columns,
        label='Rings',
    )

In [ ]:
type(model_bias_analysis_config)

In [ ]:
model_bias_monitor.create_monitoring_schedule(
    endpoint_input=endpointInput,
    ground_truth_input=ground_truth_upload_path,
    analysis_config=model_bias_analysis_config,
    output_s3_uri=model_bias_s3_report_path,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

## Create model explainability monitor

In [ ]:
df[columns_no_target].mean()

In [ ]:
[df_train[columns_no_target].mean().tolist()]

In [ ]:
df_train['Sex'].value_counts()

In [ ]:
df_train['Sex'].mode()[0]

In [ ]:
[[df_train['Sex'].mode()[0]]+
 df_train[columns_no_target[1:]].mean().tolist()]

How can you choose good baselines? Often it is desirable to select a baseline with very low information content. For example, you can construct an average instance from the training dataset by taking either the median or average for numerical features and the mode for categorical features.

In [ ]:
model_explainability_monitor = ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=sess,
    max_runtime_in_seconds=1800,
)

model_explainability_monitor_prefix = f'{prefix}/model-explainability-output'
model_explainability_baselining_job_result_uri = f's3://{bucket}/{model_explainability_monitor_prefix}/baseline'
model_explainability_s3_report_path = f's3://{bucket}/{model_explainability_monitor_prefix}/reports'

model_explainability_data_config = DataConfig(
    s3_data_input_path=val_data_s3,
    s3_output_path=model_explainability_baselining_job_result_uri,
    label='Rings',
    headers=columns,
    dataset_type='text/csv',
)

# Here use the mean value of test dataset as SHAP baseline
shap_baseline = [df_train['Sex'].mode().tolist() + 
                 df_train[columns_no_target[1:]].mean().tolist()]

shap_config = SHAPConfig(
    baseline=shap_baseline,
    num_samples=100, # val only has 418
    agg_method='mean_abs',
    save_local_shap_values=False,
)

In [ ]:
model_explainability_monitor.suggest_baseline(
    data_config=model_explainability_data_config,
    model_config=model_config,
    explainability_config=shap_config,
)
print(f'ModelExplainabilityMonitor baselining job: {model_explainability_monitor.latest_baselining_job_name}')

In [ ]:
model_explainability_monitor.latest_baselining_job.wait(logs=False)
model_explainability_constraints = model_explainability_monitor.suggested_constraints()
print()
print(f'ModelExplainabilityMonitor suggested constraints: {model_explainability_constraints.file_s3_uri}')
print(S3Downloader.read_file(model_explainability_constraints.file_s3_uri))

In [ ]:
model_explainability_analysis_config = None
if not model_explainability_monitor.latest_baselining_job:
    # Remove label because only features are required for the analysis
    model_explainability_analysis_config = ExplainabilityAnalysisConfig(
        explainability_config=shap_config,
        model_config=model_config,
        headers=columns_no_target,
    )
    
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=model_explainability_s3_report_path,
    endpoint_input=endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)